In [177]:
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
import math

In [178]:
def minute_of_angle(input,moa,input_unit="yard",output_unit="inch"):

  # One MOA is equal to 1/60 of a degree.
  moa_per_degree = moa / 60

  if input_unit == "yard":
    inches = input * 3 * 12

  if output_unit == "inch":
    # Calculate the MOA.
    moa = inches * math.radians(moa_per_degree/2) * 2

  return moa

In [179]:
def draw_centered_text(c, text, x, y, font="Helvetica", font_size=12, font_color="black"):
    c.setFont(font, font_size)
    c.setFillColor(font_color)
    text_width = c.stringWidth(text, font, font_size)
    # Assuming the height of a single line of text is roughly the font size
    text_height = font_size
    # Adjust x and y to center the text
    c.drawString(x - (text_width / 2), y - (text_height / 3), text)


In [180]:
def create_target(filename, page_size=(8.5 * inch, 11 * inch), margin=1.5 * inch, square_size=1 * inch, grid_size = 6,lines_color='black', lines_width=1, text=""):
  # Create a PDF canvas
  pdf = canvas.Canvas(filename, pagesize=page_size)
  pdf.setStrokeColor(lines_color)
  pdf.setLineWidth(lines_width)
  pdf.setLineCap(2)

  # Calculate the number of rows and columns
  num_rows = grid_size
  num_cols = grid_size

  # Calculate total grid width and height
  grid_width = num_cols * square_size
  grid_height = num_rows * square_size

  # Calculate available space for the grid
  available_width = page_size[0] - 2 * margin
  available_height = page_size[1] - 2 * margin

  # Center of page
  x_center = margin + available_width / 2
  y_center = margin + available_height / 2
  x_min = x_center - grid_width / 2
  x_max = x_center + grid_width / 2
  y_min = y_center - grid_height / 2
  y_max = y_center + grid_height / 2

  # Quadrant lines
  pdf.setStrokeColor("gray")
  pdf.setLineWidth(5)
  pdf.line(x_center, y_min, x_center, y_max)
  pdf.line(x_min, y_center, x_max, y_center)
  pdf.setStrokeColor(lines_color)
  pdf.setLineWidth(lines_width)

  # Center the grid on the page
  x_start = margin + (available_width - grid_width) / 2
  y_start = margin + (available_height - grid_height) / 2

  # Draw horizontal lines
  for i in range(num_rows + 1):
    y = y_start + i * square_size
    pdf.line(x_start, y, x_start + grid_width, y)

  # Draw vertical lines
  for i in range(num_cols + 1):
    x = x_start + i * square_size
    pdf.line(x, y_start, x, y_start + grid_height)

  # Draw diagonal lines
  pdf.setLineWidth(20)
  pdf.line(x_start, y_start, x_start + grid_width, y_start + grid_height)
  pdf.line(x_start + grid_width, y_start, x_start, y_start + grid_height)

  # Scope Adjustment Text
  scope_adjustment_text_size = 72
  draw_centered_text(pdf, "R/U", x_center-(x_center-x_min)/2, y_center-(y_center-y_min)/2, font_size=scope_adjustment_text_size, font_color="gray")
  draw_centered_text(pdf, "R/D", x_center-(x_center-x_min)/2, y_center+(y_center-y_min)/2, font_size=scope_adjustment_text_size, font_color="gray")
  draw_centered_text(pdf, "L/U", x_center+(x_center-x_min)/2, y_center-(y_center-y_min)/2, font_size=scope_adjustment_text_size, font_color="gray")
  draw_centered_text(pdf, "L/D", x_center+(x_center-x_min)/2, y_center+(y_center-y_min)/2, font_size=scope_adjustment_text_size, font_color="gray")

  pdf.setFillColor("black")
  pdf.setFont("Helvetica", 12)
  pdf.drawCentredString(x_center,0.5*inch,text)
  # Save the PDF
  pdf.save()

In [181]:
yards = 25
MOA = 1.5
fileName = f"{yards}yards_{str(MOA).replace('.','-')}moa.pdf"
gridSize = minute_of_angle(yards,MOA)
margin = 0.5 #inches
useableWidth = 8.5 - 2*margin
gridSquares = math.floor(((useableWidth/gridSize)//2)*2)
create_target(fileName,square_size=gridSize*inch,grid_size=gridSquares,text=f"{MOA} MOA grid ({gridSize:.3f} in) at {yards} yards")